<a href="https://colab.research.google.com/github/stiwari-ds/data-science-competitions/blob/main/analytics_vidhya/ml_summer_training_2022/notebooks/02_xgboost_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!pip install --upgrade xgboost
!pip install --upgrade optuna

In [87]:
import os
import gc
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import scipy.stats as st
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

import xgboost
import optuna
optuna.logging.set_verbosity(optuna.logging.ERROR)

from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from optuna.integration import XGBoostPruningCallback
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, f1_score, zero_one_loss

SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [88]:
#remove cell to run future versions
assert optuna.__version__ == '2.10.1', f'Change in Optuna version. Original notebook version: 2.10.1'
assert xgboost.__version__ == '1.6.1', f'Change in XGBoost version. Original notebook version: 1.6.1'

In [89]:
SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [90]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    GPU = True
except Exception:
    GPU = False

print(f'GPU available: {GPU}')

GPU available: True


**Data setup**

In [91]:
train_all_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/processed/all_features/train.csv'
test_all_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/processed/all_features/test.csv'

train_original_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/processed/original_filled/train.csv'
test_original_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/processed/original_filled/test.csv'

train_selected_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/processed/selected_features/train.csv'
test_selected_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/processed/selected_features/test.csv'

In [92]:
target_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/raw/train.csv'
target = pd.read_csv(target_url, usecols=['loan_default']).iloc[:, 0] #converted to series

In [113]:
test_url = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/analytics_vidhya/ml_summer_training_2022/data/raw/test.csv'
test_index = pd.read_csv(test_url, usecols=['loan_id']).iloc[:, 0]

In [93]:
train_all = pd.read_csv(train_all_url)
test_all = pd.read_csv(test_all_url)

train_original = pd.read_csv(train_original_url)
test_original = pd.read_csv(test_original_url)

train_selected = pd.read_csv(train_selected_url)
test_selected = pd.read_csv(test_selected_url)

In [94]:
train_all.head()

,age,proof_submitted,loan_amount,asset_cost,no_of_loans,no_of_curr_loans,last_delinq_none,education_fill_1,education_fill_2,age_range,loan_amount_range,asset_cost_range,no_of_loans_range,no_of_curr_loans_range,proof_is_aadhar
0,27,0,504264,820920,2,2,0,1,1,0,1,1,1,1,1
1,48,0,728556,831444,6,2,0,1,1,3,1,1,2,1,1
2,30,1,642936,826092,0,0,0,2,2,1,1,1,0,0,0
3,28,0,746556,930924,0,0,0,1,1,1,1,1,0,0,1
4,29,0,1139880,1902000,0,0,0,1,1,1,2,3,0,0,1


In [95]:
train_original.head()

,age,education_fill_1,proof_submitted,loan_amount,asset_cost,no_of_loans,no_of_curr_loans,last_delinq_none
0,27,1,0,504264,820920,2,2,0
1,48,1,0,728556,831444,6,2,0
2,30,2,1,642936,826092,0,0,0
3,28,1,0,746556,930924,0,0,0
4,29,1,0,1139880,1902000,0,0,0


In [96]:
train_selected.head()

,age,proof_submitted,loan_amount,asset_cost,no_of_loans,no_of_curr_loans
0,27,0,504264,820920,2,2
1,48,0,728556,831444,6,2
2,30,1,642936,826092,0,0
3,28,0,746556,930924,0,0
4,29,0,1139880,1902000,0,0


In [97]:
target.head()

0    0
1    0
2    1
3    0
4    0
Name: loan_default, dtype: int64

**Categorical data conversion**

In [98]:
categorical_features = ['proof_submitted', 'last_delinq_none', 
                        'education_fill_1', 'education_fill_2', 
                        'age_range', 'loan_amount_range', 
                        'asset_cost_range', 'no_of_loans_range', 
                        'no_of_curr_loans_range', 'proof_is_aadhar']

In [99]:
train_all[categorical_features] = train_all[categorical_features].astype('category')
test_all[categorical_features] = test_all[categorical_features].astype('category')

In [100]:
cat_cols = [f for f in train_original.columns if f in categorical_features]

train_original[cat_cols] = train_original[cat_cols].astype('category')
test_original[cat_cols] = test_original[cat_cols].astype('category')

In [101]:
cat_cols = [f for f in train_selected.columns if f in categorical_features]

train_selected[cat_cols] = train_selected[cat_cols].astype('category')
test_selected[cat_cols] = test_selected[cat_cols].astype('category')

# Baselines

In [102]:
def fit_baseline(X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, shuffle=True, stratify=target, random_state=SEED)
    
    baseline = XGBClassifier(tree_method='gpu_hist', enable_categorical=True)
    baseline.fit(
        X_train, y_train, 
        eval_set=[(X_val, y_val)],
        verbose=0)
    
    preds = baseline.predict(X_val)
    print(classification_report(y_val, preds))

**Full data: original + created features**

In [103]:
fit_baseline(train_all, target)

              precision    recall  f1-score   support

           0       0.63      0.75      0.68       840
           1       0.48      0.34      0.40       560

    accuracy                           0.59      1400
   macro avg       0.55      0.55      0.54      1400
weighted avg       0.57      0.59      0.57      1400



**Original features**

In [104]:
fit_baseline(train_original, target)

              precision    recall  f1-score   support

           0       0.61      0.72      0.66       840
           1       0.43      0.31      0.36       560

    accuracy                           0.56      1400
   macro avg       0.52      0.52      0.51      1400
weighted avg       0.54      0.56      0.54      1400



**Selected features**

In [105]:
fit_baseline(train_selected, target)

              precision    recall  f1-score   support

           0       0.62      0.76      0.69       840
           1       0.47      0.31      0.38       560

    accuracy                           0.58      1400
   macro avg       0.54      0.54      0.53      1400
weighted avg       0.56      0.58      0.56      1400



# Hyperparameter tuning

In [106]:
def objective(trial, data, base_params):

    scores = []
    X, y = data
    cat_features = list(X.select_dtypes(include='category').columns)

    param_grid = {
        'objective': trial.suggest_categorical(
            'objective', ['binary:logistic', 'binary:hinge']),
        'learning_rate': trial.suggest_float(
            'learning_rate', 0.01, 0.4, step=0.01),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float(
            'colsample_bytree', 0.5, 1.0, step=0.05),
        'colsample_bylevel': trial.suggest_float(
            'colsample_bylevel', 0.5, 1.0, step=0.05),
        'colsample_bynode': trial.suggest_float(
            'colsample_bynode', 0.5, 1.0, step=0.05),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
        'gamma': trial.suggest_float('gamma', 0, 20),
        'alpha': trial.suggest_float('alpha', 1e-5, 1e2, log=True),
        'lambda': trial.suggest_float('lambda', 1e-5, 1e2, log=True),
        'scale_pos_weight': trial.suggest_float(
            'scale_pos_weight', 1, 1.5, step=0.05)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        model = XGBClassifier(
            **base_params, 
            **param_grid,
            callbacks=[XGBoostPruningCallback(
                trial=trial, 
                observation_key='validation_0-error')]
        )
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        preds = model.predict(X_val)
        scores.append(f1_score(y_val, preds, average='macro'))
    
    return np.mean(scores)

In [107]:
def tune_params(data, base_params, n_trials=10, direction='maximize'):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction)
    
    study.optimize(
        func=lambda trial: objective(trial, data, base_params),
        n_trials=n_trials,
        gc_after_trial=True)
    
    return study

# Cross-validation

In [108]:
def evaluate_model(data, model_params, verbose=True):
    
    preds_test = []
    scores_f1 = [] #validation set weighted-f1 scores
    scores_0_1_loss = [] #validation set zero-one loss
    
    X, X_test, y = data
    cat_features = list(X.select_dtypes(include='category').columns)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = XGBClassifier(**model_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        
        preds_val = model.predict(X_val)
        preds_test.append(model.predict(X_test))
        
        scores_0_1_loss.append(zero_one_loss(y_val, preds_val))
        scores_f1.append(f1_score(y_val, preds_val, average='macro'))
    
    if verbose:
        scores_df = pd.DataFrame.from_dict({
            '0-1 loss': scores_0_1_loss,
            'Macro-f1': scores_f1
        })
        scores_df.index.name = 'Fold'
        display(scores_df)
    
    print(f'Average 0-1 loss (error%) = {np.mean(scores_0_1_loss):.4f} (with std = {np.std(scores_0_1_loss):.4f})')
    print(f'Average Macro-f1 = {np.mean(scores_f1):.4f} (with std = {np.std(scores_f1):.4f})')
    
    preds_test = st.mode(np.column_stack(preds_test), axis=1).mode
    return preds_test

In [109]:
def run_experiment(data, n_trials=5):
    """Tune the hyperparameters, train and evaluate the model, and return test predictions."""
    
    X, X_test, y = data
    
    base_params = {
        'n_estimators': 2000,
        'eval_metric': 'error',
        'early_stopping_rounds': 50,
        'tree_method': 'gpu_hist' if GPU else 'hist', 
        'enable_categorical': GPU,
        'max_cat_to_onehot': 5, #internal one-hot encoding
        'use_best_model': True,
        'verbosity': 0,
        'random_seed': SEED
    }
    
    print(f'---------------Hyperparameter tuning---------------')
    study = tune_params(
        data=(X, y), 
        base_params=base_params,
        n_trials=n_trials,
        direction='maximize')
    print(f'Best trial: {study.best_trial.number} -> Best value (f1-macro): {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:20} - {v}')
    
    model_params = {**base_params, **study.best_params}
    print(f'-----------------Cross-validation------------------')
    preds_test = evaluate_model(
        data=(X, X_test, y), 
        model_params=model_params)
    
    return preds_test

In [110]:
%%time
preds_all = run_experiment((train_all, test_all, target), 500)

---------------Hyperparameter tuning---------------
Best trial: 386 -> Best value (f1-macro): 0.57081
Best hyperparameters:
objective            - binary:hinge
learning_rate        - 0.02
max_depth            - 5
subsample            - 0.7
colsample_bytree     - 1.0
colsample_bylevel    - 0.9
colsample_bynode     - 0.9
min_child_weight     - 6
gamma                - 1.5124227259932637
alpha                - 0.0012085252276947717
lambda               - 14.126618915538158
scale_pos_weight     - 1.2
-----------------Cross-validation------------------


,0-1 loss,Macro-f1
Fold,,
0,0.4064,0.5824
1,0.4100,0.5606
2,0.4100,0.5677
3,0.4007,0.5701
4,0.4071,0.5733


Average 0-1 loss (error%) = 0.4069 (with std = 0.0034)
Average Macro-f1 = 0.5708 (with std = 0.0071)
CPU times: user 12min 30s, sys: 12.4 s, total: 12min 42s
Wall time: 11min 19s


In [111]:
%%time
preds_original = run_experiment((train_original, test_original, target), 500)

---------------Hyperparameter tuning---------------
Best trial: 297 -> Best value (f1-macro): 0.57296
Best hyperparameters:
objective            - binary:hinge
learning_rate        - 0.02
max_depth            - 19
subsample            - 0.9
colsample_bytree     - 1.0
colsample_bylevel    - 0.75
colsample_bynode     - 0.95
min_child_weight     - 9
gamma                - 2.8757718442132703
alpha                - 0.5934285811858028
lambda               - 0.0007744778772270202
scale_pos_weight     - 1.15
-----------------Cross-validation------------------


,0-1 loss,Macro-f1
Fold,,
0,0.3993,0.5791
1,0.4043,0.5813
2,0.4143,0.5700
3,0.4050,0.5675
4,0.4057,0.5669


Average 0-1 loss (error%) = 0.4057 (with std = 0.0048)
Average Macro-f1 = 0.5730 (with std = 0.0060)
CPU times: user 19min 16s, sys: 13.8 s, total: 19min 30s
Wall time: 18min 15s


In [112]:
%%time
preds_selected = run_experiment((train_selected, test_selected, target), 500)

---------------Hyperparameter tuning---------------
Best trial: 235 -> Best value (f1-macro): 0.56384
Best hyperparameters:
objective            - binary:hinge
learning_rate        - 0.02
max_depth            - 16
subsample            - 0.6
colsample_bytree     - 0.9
colsample_bylevel    - 0.9
colsample_bynode     - 1.0
min_child_weight     - 5
gamma                - 2.233161138314077
alpha                - 0.0737242140960301
lambda               - 0.564222168956388
scale_pos_weight     - 1.15
-----------------Cross-validation------------------


,0-1 loss,Macro-f1
Fold,,
0,0.4050,0.5707
1,0.4157,0.5611
2,0.4286,0.5630
3,0.3950,0.5657
4,0.4200,0.5586


Average 0-1 loss (error%) = 0.4129 (with std = 0.0117)
Average Macro-f1 = 0.5638 (with std = 0.0042)
CPU times: user 11min 19s, sys: 11.3 s, total: 11min 30s
Wall time: 10min 26s


# Generating submissions

In [117]:
sub_all = pd.DataFrame({
    'loan_id': test_index,
    'loan_default': preds_all.ravel()
})
sub_all.to_csv('sub_all.csv', index=False)

In [118]:
sub_original = pd.DataFrame({
    'loan_id': test_index,
    'loan_default': preds_original.ravel()
})
sub_original.to_csv('sub_original.csv', index=False)

In [119]:
sub_selected = pd.DataFrame({
    'loan_id': test_index,
    'loan_default': preds_selected.ravel()
})
sub_selected.to_csv('sub_selected.csv', index=False)